# Template File: Linearization

In [1]:
import sympy
from sympy import *
from sympy.physics.vector.printing import vlatex
from IPython.display import Math, display

init_printing()

def dotprint(expr):
    display(Math(vlatex(expr)))

In [2]:
t = symbols('t')
theta = symbols(r'\theta', cls=Function)
theta = theta(t)
theta

theta_dot = theta.diff(t)
theta_ddot = theta.diff(t,2)

ell, m, g, P_0, tau, b = symbols(r'\ell, m, g, P_0, \tau, b', real=True)

In [13]:
LHS = Rational(1,3)*m*ell**2*theta_ddot + Rational(1,2)*m*g*ell*cos(theta)
RHS = tau - b*theta_dot

dynamics = Eq(LHS, RHS)
dotprint(dynamics)

<IPython.core.display.Math object>

## Deriving Nonlinear State Space Equations

Solve for our highest derivative, $\ddot{\theta}$:

In [4]:
solve_dict = solve(dynamics, (theta_ddot,), simplify=True, dict=True)[0]
dotprint(solve_dict)

<IPython.core.display.Math object>

Create our state vector $x = [x_1, x_2]$:

In [5]:
state = MatrixSymbol('x', 2,1)
dotprint(state)

<IPython.core.display.Math object>

In [6]:
theta_ddot_expr = solve_dict[theta_ddot]
state_deriv = Matrix([theta_dot, theta_ddot_expr])
dotprint(state_deriv)

<IPython.core.display.Math object>

Now we can substitute in our $x_1, x_2$ values in!

In [7]:
# Dictionary for substitutions
subs_dict = {
    theta: state[0],
    theta_dot: state[1],
    
}

f_expr = state_deriv.subs(subs_dict)
dotprint(f_expr)

<IPython.core.display.Math object>

We now have our nonlinear state space equations!

## Linearization

First, we need to find an equilibrium point. We can either do this by hand or use Sympy's `solve` function to do this.

**Note:** It's a good idea to practice doing this by hand in addition to using Sympy!

In [8]:
# This equation represents f(x,u) = 0
equilibrium_equation = Eq(f_expr, Matrix([0,0]))

eq_solve_dict = solve(equilibrium_equation, (state[0], state[1], tau), simplify=True, dict=True)[0]
dotprint(eq_solve_dict)

<IPython.core.display.Math object>

In [9]:
u_eq = eq_solve_dict[tau]
theta_dot_eq = eq_solve_dict[state[1]]

Excellent! We have equilibrium solutions for $u = \tau$ and $x_2 = \dot{\theta}$. 

But think for a second: Why didn't Sympy give us an equilibrium solution for $x_1 = \theta$?

#### Define A, B Jacobians

We can use Sympy's `jacobian` function to find the jacobians of `f(x,u)`.

First we find $A = \frac{\partial f}{\partial x}$:

In [10]:
A = f_expr.jacobian(state)
dotprint(A)

<IPython.core.display.Math object>

We can evaluate at our specific $(x_e, u_e)$ point using `subs`.

Since $\theta$ can be "anything", we choose it and compute everything else based off of that. Here we'll choose $\theta = 0$.

In [11]:
theta_desired = 0

A_eq = A.subs({state[0]: theta_desired})
dotprint(A_eq)

<IPython.core.display.Math object>

Now we do a similar process to find $B = \frac{\partial f}{\partial u}$

In [12]:
B = f_expr.jacobian([tau])
dotprint(B)

<IPython.core.display.Math object>

Since B is not a function of the states or control inputs _in this case_, we don't have to substitute. But in general we would have to substitute in values!

Now we are ready to make our final equations of motion.